In [1]:
import numpy as np

In [2]:
cartesian_position = np.random.uniform(-1, 1, (10, 2))
cartesian_position

array([[ 0.62242298, -0.95687436],
       [-0.28946166, -0.19573368],
       [-0.95488804, -0.00772582],
       [-0.86441419, -0.27353684],
       [-0.31352347, -0.27097979],
       [ 0.91911551,  0.17220942],
       [-0.22700369, -0.85146781],
       [-0.82652134, -0.36068693],
       [-0.93015085, -0.48976778],
       [ 0.61446708,  0.69682854]])

In [3]:
assert cartesian_position.shape[-1] == 2
assert len(cartesian_position.shape) <= 2

In [4]:
# cartesian_position = cartesian_position[0, :]
# cartesian_position

In [5]:
if len(cartesian_position.shape) == 1:
    cartesian_position = cartesian_position[np.newaxis]

cartesian_position

array([[ 0.62242298, -0.95687436],
       [-0.28946166, -0.19573368],
       [-0.95488804, -0.00772582],
       [-0.86441419, -0.27353684],
       [-0.31352347, -0.27097979],
       [ 0.91911551,  0.17220942],
       [-0.22700369, -0.85146781],
       [-0.82652134, -0.36068693],
       [-0.93015085, -0.48976778],
       [ 0.61446708,  0.69682854]])

In [6]:
polar_position = np.zeros_like(cartesian_position)
polar_position[:, 0] = np.linalg.norm(cartesian_position[:, :], axis=1)
polar_position

array([[1.14149854, 0.        ],
       [0.34942771, 0.        ],
       [0.95491929, 0.        ],
       [0.90666107, 0.        ],
       [0.41439958, 0.        ],
       [0.9351093 , 0.        ],
       [0.88120832, 0.        ],
       [0.9017941 , 0.        ],
       [1.05121505, 0.        ],
       [0.92905318, 0.        ]])

In [7]:
polar_position[:, 1] = np.arctan2(cartesian_position[:, 1], cartesian_position[:, 0])
polar_position

array([[ 1.14149854, -0.99408716],
       [ 0.34942771, -2.54701975],
       [ 0.95491929, -3.13350202],
       [ 0.90666107, -2.83512066],
       [ 0.41439958, -2.42885241],
       [ 0.9351093 ,  0.18521685],
       [ 0.88120832, -1.83133908],
       [ 0.9017941 , -2.73011296],
       [ 1.05121505, -2.65693399],
       [ 0.92905318,  0.84812499]])

In [8]:
def convert_to_polar(cartesian_position: np.ndarray) -> np.ndarray:
    """
    Convert given 2D position array from cartesian to polar coordinates
    """
    assert cartesian_position.shape[-1] == 2, "Expected 2D cartesian coordinates"
    assert len(cartesian_position.shape) <= 2,  "Expected array of 2D cartesian coordinates"

    if len(cartesian_position.shape) == 1:
        cartesian_position = cartesian_position[np.newaxis]
    
    polar_position = np.zeros_like(cartesian_position)
    polar_position[:, 0] = np.linalg.norm(cartesian_position[:, :], axis=1)
    polar_position[:, 1] = np.arctan2(cartesian_position[:, 1], cartesian_position[:, 0])
    
    return polar_position

In [9]:
known_tags = {
    "A": np.array((1.0, 2.0, 0.0)),
    "B": np.array((-1.0, -2.0, 0.0)),
    "C": np.array((2.0, 2.0, 0.0))
}

detected_tags = {
    "A": np.array((1.0, 2.0, 0.0)),
    "B": np.array((-1.0, -2.0, 0.0)),
    "C": np.array((2.0, 2.0, 0.0))
}

In [10]:
if len(detected_tags.keys()):
    known_tag_positions = np.zeros((len(detected_tags.keys()), 3))
    measured_tag_positions = np.zeros((len(detected_tags.keys()), 3))

    for index, tag_name in enumerate(detected_tags.keys()):
        known_tag_positions[index, :] = np.array(known_tags[tag_name])
        measured_tag_positions[index, :] = detected_tags[tag_name]

    # Now a column matrix of tags
    known_tags_polar = convert_to_polar(known_tag_positions[:, :-1]).T
    measured_tags_polar = convert_to_polar(measured_tag_positions[:, :-1]).T


In [11]:
known_tag_positions

array([[ 1.,  2.,  0.],
       [-1., -2.,  0.],
       [ 2.,  2.,  0.]])

In [12]:
measured_tag_positions

array([[ 1.,  2.,  0.],
       [-1., -2.,  0.],
       [ 2.,  2.,  0.]])

In [13]:
known_tags_polar

array([[ 2.23606798,  2.23606798,  2.82842712],
       [ 1.10714872, -2.03444394,  0.78539816]])

In [14]:
state = np.random.uniform(-1, 1, (3, 1))
state

array([[-0.58800443],
       [-0.38909288],
       [ 0.65330832]])

In [15]:
predicted_tags = np.zeros_like(measured_tags_polar)
predicted_tags

array([[0., 0., 0.],
       [0., 0., 0.]])

In [16]:
predicted_tags[1, :] = known_tags_polar[1, :] - state[-1]
predicted_tags[0, :] = known_tags_polar[0, :] - (state[0] * np.cos(known_tags_polar[1, :]) + state[1] * np.sin(known_tags_polar[1, :]))
predicted_tags

array([[ 2.8470468 ,  1.62508915,  3.51933926],
       [ 0.4538404 , -2.68775225,  0.13208985]])

In [17]:
state = np.zeros_like(state)
state[0] = 0.1
state[1] = 0.1
state

array([[0.1],
       [0.1],
       [0. ]])

In [18]:
x, y, theta = state.flatten()
r_i, alpha_i = measured_tags_polar[0,:], measured_tags_polar[1,:]
r_j, alpha_j = known_tags_polar[0,:], known_tags_polar[1,:]

In [19]:
predicted_tags = np.array([r_j-(x *np.cos(alpha_j)+ y*np.sin(alpha_j)), (alpha_j-theta) ]).reshape(2,-1)
predicted_tags

array([[ 2.1019039 ,  2.37023206,  2.68700577],
       [ 1.10714872, -2.03444394,  0.78539816]])

In [20]:
predicted_tags_cartesian = np.zeros_like(predicted_tags)
predicted_tags_cartesian[0] = predicted_tags[0] * np.cos(predicted_tags[1])
predicted_tags_cartesian[1] = predicted_tags[0] * np.sin(predicted_tags[1])
predicted_tags_cartesian

array([[ 0.94, -1.06,  1.9 ],
       [ 1.88, -2.12,  1.9 ]])

In [21]:
known_tags_polar

array([[ 2.23606798,  2.23606798,  2.82842712],
       [ 1.10714872, -2.03444394,  0.78539816]])

In [22]:
measured_tags_polar

array([[ 2.23606798,  2.23606798,  2.82842712],
       [ 1.10714872, -2.03444394,  0.78539816]])

In [23]:
v_t = measured_tags_polar - np.array([r_j-(x *np.cos(alpha_j)+ y*np.sin(alpha_j)), (alpha_j-theta)]).reshape(2,-1)
v_t

array([[ 0.13416408, -0.13416408,  0.14142136],
       [ 0.        ,  0.        ,  0.        ]])

In [24]:
known_tags_polar.flatten()

array([ 2.23606798,  2.23606798,  2.82842712,  1.10714872, -2.03444394,
        0.78539816])

In [25]:
measured_tags_polar

array([[ 2.23606798,  2.23606798,  2.82842712],
       [ 1.10714872, -2.03444394,  0.78539816]])

In [26]:
estimated_pos = np.random.uniform(-1, 1, (3, 1))
estimated_pos

array([[0.22162558],
       [0.50705809],
       [0.24945403]])

In [27]:
x, y, theta = estimated_pos.flatten()
print(x, y, theta)

0.22162558359292706 0.5070580883260347 0.24945402873799738


# Testing loop

### Prediction update

In [233]:
state = np.array((1.0, 1.0, np.pi/4))[np.newaxis].T
# state = np.array((0.0, 0.0, np.pi/2))
state

array([[1.        ],
       [1.        ],
       [0.78539816]])

In [234]:
control_input = np.array((0.5, 0.5, 0.1))[np.newaxis].T
control_input

array([[0.5],
       [0.5],
       [0.1]])

In [235]:
F_k_1 = np.eye(3)
F_k_1

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [236]:
time_step = 0.5
G_k_1 = np.zeros((3,3))
np.fill_diagonal(G_k_1, time_step)
display(G_k_1)

array([[0.5, 0. , 0. ],
       [0. , 0.5, 0. ],
       [0. , 0. , 0.5]])

In [237]:
x_k = F_k_1 @ state
display(x_k)
x_k = G_k_1 @ control_input
display(x_k)
x_k = F_k_1 @ state + G_k_1 @ control_input
x_k.flatten()

array([[1.        ],
       [1.        ],
       [0.78539816]])

array([[0.25],
       [0.25],
       [0.05]])

array([1.25      , 1.25      , 0.83539816])

In [238]:
cov_matrix = np.eye(3) * 0.01
display(cov_matrix)
pred_noise_density = np.eye(3) * 0.01
display(pred_noise_density)

array([[0.01, 0.  , 0.  ],
       [0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.01]])

array([[0.01, 0.  , 0.  ],
       [0.  , 0.01, 0.  ],
       [0.  , 0.  , 0.01]])

In [239]:
P_k = F_k_1 @ cov_matrix @ F_k_1.T + pred_noise_density
display(P_k)

cov_matrix = P_k

array([[0.02, 0.  , 0.  ],
       [0.  , 0.02, 0.  ],
       [0.  , 0.  , 0.02]])

### Measurement update

In [210]:
known_tags = {
    "A": np.array((1.0, 2.0, 0.0)),
    "B": np.array((-1.0, -2.0, 0.0)),
    "C": np.array((2.0, 2.0, 0.0))
}

detected_tags = {
    "A": np.array((1.0, 2.0, 0.0)),
    "B": np.array((-1.0, -2.0, 0.0)),
    "C": np.array((2.0, 2.0, 0.0))
}

dim = 2
num_tags = len(detected_tags.keys())
num_tags

3

In [184]:
if len(detected_tags.keys()):
    known_tag_positions = np.zeros((len(detected_tags.keys()), 3))
    measured_tag_positions = np.zeros((len(detected_tags.keys()), 3))

    for index, tag_name in enumerate(detected_tags.keys()):
        known_tag_positions[index, :] = np.array(known_tags[tag_name])
        measured_tag_positions[index, :] = detected_tags[tag_name]

    # Now a column matrix of tags
    known_tags_polar = convert_to_polar(known_tag_positions[:, :-1]).T
    measured_tags_polar = convert_to_polar(measured_tag_positions[:, :-1]).T

In [185]:
x, y, theta = state.flatten()
print(x, y, theta)
# Measured tags_polar
r_i, alpha_i = measured_tags_polar[0,:], measured_tags_polar[1,:]
print(r_i, alpha_i)
# Known tags
r_j, alpha_j = known_tags_polar[0,:], known_tags_polar[1,:]
print(r_j, alpha_j)

1.0 1.0 0.7853981633974483
[2.23606798 2.23606798 2.82842712] [ 1.10714872 -2.03444394  0.78539816]
[2.23606798 2.23606798 2.82842712] [ 1.10714872 -2.03444394  0.78539816]


In [186]:
predicted_tags = np.array([r_j-(x *np.cos(alpha_j)+ y*np.sin(alpha_j)), (alpha_j-theta)]).reshape(2,-1)
predicted_tags

array([[ 0.89442719,  3.57770876,  1.41421356],
       [ 0.32175055, -2.8198421 ,  0.        ]])

In [187]:
predicted_tags_cartesian = np.zeros_like(predicted_tags)
predicted_tags_cartesian[0] = predicted_tags[0] * np.cos(predicted_tags[1])
predicted_tags_cartesian[1] = predicted_tags[0] * np.sin(predicted_tags[1])
predicted_tags_cartesian

array([[ 0.84852814, -3.39411255,  1.41421356],
       [ 0.28284271, -1.13137085,  0.        ]])

In [188]:
tag_noise = np.random.uniform(-0.25, 0.25, predicted_tags.shape)
tag_noise

array([[-0.24221121, -0.09267933,  0.20257584],
       [ 0.18007196,  0.2144071 ,  0.0426121 ]])

In [189]:
measured_tags_polar = predicted_tags + tag_noise
measured_tags_polar

array([[ 0.65221598,  3.48502944,  1.61678941],
       [ 0.50182251, -2.605435  ,  0.0426121 ]])

In [190]:
# Measured tags
r_i, alpha_i = measured_tags_polar[0,:], measured_tags_polar[1,:]
print(r_i, alpha_i)
# Known tags
r_j, alpha_j = known_tags_polar[0,:], known_tags_polar[1,:]
print(r_j, alpha_j)

[0.65221598 3.48502944 1.61678941] [ 0.50182251 -2.605435    0.0426121 ]
[2.23606798 2.23606798 2.82842712] [ 1.10714872 -2.03444394  0.78539816]


In [191]:
v_t = measured_tags_polar - \
            np.array([r_j - (x * np.cos(alpha_j) + y * np.sin(alpha_j)), (alpha_j - theta)]).reshape(2,-1)
v_t

array([[-0.24221121, -0.09267933,  0.20257584],
       [ 0.18007196,  0.2144071 ,  0.0426121 ]])

Turns out the shapes were wrong, correcting them below

In [211]:
if len(detected_tags.keys()):
    known_tag_positions = np.zeros((len(detected_tags.keys())*2, 1))
    measured_tag_positions = np.zeros((len(detected_tags.keys())*2, 1))

    for index, tag_name in enumerate(detected_tags.keys()):
        known_tag_positions[index*2:index*2+2, :] = np.array(known_tags[tag_name])[:2][np.newaxis].T
        measured_tag_positions[index*2:index*2+2, :] = detected_tags[tag_name][:2][np.newaxis].T

    display(known_tag_positions)
    display(measured_tag_positions)
    # Now a column matrix of tags
    # known_tags_polar = convert_to_polar(known_tag_positions[:, :-1]).T
    # measured_tags_polar = convert_to_polar(measured_tag_positions[:, :-1]).T

array([[ 1.],
       [ 2.],
       [-1.],
       [-2.],
       [ 2.],
       [ 2.]])

array([[ 1.],
       [ 2.],
       [-1.],
       [-2.],
       [ 2.],
       [ 2.]])

In [212]:
known_tags_polar = np.zeros(known_tag_positions.shape)
for index in range(num_tags):
    known_tags_polar[dim*index:dim*index+2] = convert_to_polar(known_tag_positions[dim*index:dim*index+2].flatten()).T

known_tags_polar

array([[ 2.23606798],
       [ 1.10714872],
       [ 2.23606798],
       [-2.03444394],
       [ 2.82842712],
       [ 0.78539816]])

Simulating a measurement of tags - in column form this time

In [213]:
predicted_tags = np.zeros((dim*num_tags, 1))

for index in range(num_tags):
    alpha_j = known_tags_polar[dim*index+1][0]
    r_j = known_tags_polar[dim*index][0]

    predicted_tags[dim*index:dim*index+2] = np.array([
        [r_j - (x * np.cos(alpha_j) + y * np.sin(alpha_j))],
        [alpha_j - theta],
    ])

predicted_tags

array([[ 0.89442719],
       [ 0.32175055],
       [ 3.57770876],
       [-2.8198421 ],
       [ 1.41421356],
       [ 0.        ]])

In [214]:
measured_tags = predicted_tags + np.random.uniform(-0.5, 0.5, predicted_tags.shape)
measured_tags

array([[ 0.52053003],
       [ 0.70686725],
       [ 3.87505263],
       [-2.94369103],
       [ 1.19828123],
       [ 0.38768556]])

In [215]:
v_t = measured_tags - predicted_tags
display(v_t)
display(v_t.shape)

array([[-0.37389716],
       [ 0.38511669],
       [ 0.29734387],
       [-0.12384893],
       [-0.21593233],
       [ 0.38768556]])

(6, 1)

Jacobian

In [216]:
H = np.array([[np.zeros_like(alpha_j), np.zeros_like(alpha_j), -np.ones_like(alpha_j)], 
                       [-np.cos(alpha_j), -np.sin(alpha_j), np.zeros_like(alpha_j)]])
display(H)
display(H.shape)

array([[ 0.        ,  0.        , -1.        ],
       [-0.70710678, -0.70710678,  0.        ]])

(2, 3)

In [217]:
H = np.zeros((2*num_tags, 3))
display(H.shape)

for index in range(num_tags):
    print(index)
    alpha_j = known_tags_polar[dim*index+1][0]
    r_j = known_tags_polar[dim*index][0]

    H_tag = np.array([
        [0, 0, -1],
        [-np.cos(alpha_j), -np.sin(alpha_j), 0]
    ])
    
    display(H_tag)
    H[dim*index:(dim*index)+2, :] = H_tag

display(H)

(6, 3)

0


array([[ 0.        ,  0.        , -1.        ],
       [-0.4472136 , -0.89442719,  0.        ]])

1


array([[ 0.        ,  0.        , -1.        ],
       [ 0.4472136 ,  0.89442719,  0.        ]])

2


array([[ 0.        ,  0.        , -1.        ],
       [-0.70710678, -0.70710678,  0.        ]])

array([[ 0.        ,  0.        , -1.        ],
       [-0.4472136 , -0.89442719,  0.        ],
       [ 0.        ,  0.        , -1.        ],
       [ 0.4472136 ,  0.89442719,  0.        ],
       [ 0.        ,  0.        , -1.        ],
       [-0.70710678, -0.70710678,  0.        ]])

In [245]:
# TODO: Add measurement_cov here
measurement_cov = np.eye(6) * 0.01

In [246]:
sigma = H @ cov_matrix @ H.T + measurement_cov
sigma

array([[ 0.03      ,  0.        ,  0.02      ,  0.        ,  0.02      ,
         0.        ],
       [ 0.        ,  0.03      ,  0.        , -0.02      ,  0.        ,
         0.01897367],
       [ 0.02      ,  0.        ,  0.03      ,  0.        ,  0.02      ,
         0.        ],
       [ 0.        , -0.02      ,  0.        ,  0.03      ,  0.        ,
        -0.01897367],
       [ 0.02      ,  0.        ,  0.02      ,  0.        ,  0.03      ,
         0.        ],
       [ 0.        ,  0.01897367,  0.        , -0.01897367,  0.        ,
         0.03      ]])

Kalman gain

In [247]:
def kalman_filter_gain (
        cov_matrix : np.ndarray,
        measurement_matrix: np.ndarray,
        sigma : np.ndarray
        ):

    kalman_gain = cov_matrix @ measurement_matrix.T @ np.linalg.pinv(sigma)

    return kalman_gain

In [248]:
kalman_gain = kalman_filter_gain(cov_matrix, H, sigma)
display(kalman_gain)
display(kalman_gain.shape)

array([[-3.18495282e-18, -1.28677524e-17, -2.01649491e-16,
        -1.95811242e-16,  2.06408902e-16, -4.71404521e-01],
       [ 1.79873262e-17, -3.44010458e-01, -1.82315462e-16,
         3.44010458e-01,  1.55298435e-16, -3.62618862e-02],
       [-2.85714286e-01,  2.32757878e-16, -2.85714286e-01,
         1.61703604e-16, -2.85714286e-01, -7.10129235e-17]])

(3, 6)

Estimation

In [249]:
display(state)
new_state = state + kalman_gain @ v_t
display(new_state)

array([[1.        ],
       [1.        ],
       [0.78539816]])

array([[0.81724327],
       [0.81085229],
       [0.86896549]])

In [250]:
display(cov_matrix)
new_cov_matrix = cov_matrix - kalman_gain @ sigma @ kalman_gain.T
display(new_cov_matrix)

array([[0.02, 0.  , 0.  ],
       [0.  , 0.02, 0.  ],
       [0.  , 0.  , 0.02]])

array([[ 1.33333333e-02, -6.66666667e-03, -3.37256497e-19],
       [-6.66666667e-03,  7.17948718e-03,  8.61890827e-20],
       [-3.37256497e-19,  8.61890827e-20,  2.85714286e-03]])